# Peretemuan 6
- Contour - Part 2
    - Contour Detection & Application
    - CUDA Contour Detection 
- Hough Transform
    - Hough line
    - Hough Circle
    - Hough Line Application
    - Hough Transform CUDA

___
### Maximizing Jetson Nano Perfomance

In [ ]:
# sudo nvpmodel -m 0
# sudo jetson_clocks

In [ ]:
import cv2 
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
# check OpenCV Version

cv2.__version__

In [ ]:
# draw_ped() function to draw bounding box with top labeled text


def draw_ped(img, label, x0, y0, xt, yt, font_size=0.4, alpha=0.5, bg_color=(255,0,0), ouline_color=(255,255,255), text_color=(0,0,0)):
    overlay = np.zeros_like(img)

    y0, yt = max(y0 - 15, 0) , min(yt + 15, img.shape[0])
    x0, xt = max(x0 - 15, 0) , min(xt + 15, img.shape[1])

    (w, h), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_size, 1)
    cv2.rectangle(overlay,
                    (x0, y0 + baseline),  
                    (max(xt, x0 + w), yt), 
                    bg_color, 
                    -1)
    cv2.rectangle(img,
                    (x0, y0 + baseline),  
                    (max(xt, x0 + w), yt), 
                    ouline_color, 
                    2)
    pts = np.array([[x0, y0 - h - baseline], # top left
                    [x0 + w, y0 - h - baseline], # top right
                    [x0 + w + 10, y0 + baseline], # bolom right
                    [x0,y0 + baseline]]) # bottom left
    cv2.fillPoly(img, [pts], ouline_color) # add label white fill
    cv2.polylines(img, [pts], True, ouline_color, 2) # add label white border 
    cv2.putText(img, 
                label, 
                (x0, y0),                   
                cv2.FONT_HERSHEY_SIMPLEX,     
                font_size,                          
                text_color,                
                1,
                cv2.LINE_AA) 

    img_blend = cv2.addWeighted(img, 1, overlay, alpha, 0.0)

    return img_blend

___
# 1. Contour - Part 2
## 1.1 Contour Detection & Application
## 1.1.1 Visual Inspection based on Simple Threshold 
- Image sample :</br>
<img src="part_b.jpg" width=200></img></br></br>

In [ ]:
# Example 1 : (part_b.jpg)
# Find Contour using Simple Thresholding + Otsu's

img = cv2.imread("part_b.jpg")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

__, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

for cnt in contours : 
    cv2.drawContours(img, cnt, -1, (0,0,255), 3)

plt.figure(figsize=(20,7))
plt.subplot(1,2,1)
plt.imshow(thresh, cmap="gray")
plt.title("Binary Image")

plt.subplot(1,2,2)
plt.imshow(img[:,:,::-1])
plt.title("Result")

In [ ]:
# As you can see, Simple thresholding using Otsu's method giving us not a good result.
# This is happen because the original image has light ilumination in it.
# the option is using using Range Thresholding (based color)

___
## 1.1.2 Visual Inspection Based On Range Thresholding

In [ ]:
# convert RGB to HSV

rgb = np.array([[[136,119,114]]], np.uint8)

hsv=cv2.cvtColor(rgb, cv2.COLOR_BGR2HSV)

hsv

In [ ]:
# define range of gray color in HSV
lower_gray = np.array([0, 0, 20])
upper_gray = np.array([180, 100, 150])

In [ ]:
# EXAMPLE 2 : (part_b.jpg)
# Detect Contour from Binary Image (Range Thresholding)

img = cv2.imread("part_b.jpg")

# convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Threshold the HSV image to get only gray colors
mask = cv2.inRange(hsv.copy(), lower_gray, upper_gray)

contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

for cnt in contours : 
    cv2.drawContours(img, cnt, -1, (0,0,0), 3)

plt.figure(figsize=(20,14))
plt.subplot(1,2,1)
plt.imshow(mask, cmap="gray")
plt.title("Binary Image")

plt.subplot(1,2,2)
plt.imshow(img[:,:,::-1])
plt.title("Result")

In [ ]:
# EXAMPLE 3 : (part_b.jpg)
# Detect Contour from Binary Image (Range Thresholding) + 
# Background removal

img = cv2.imread("part_b.jpg")

#convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Threshold the HSV image to get only gray colors
mask = cv2.inRange(hsv.copy(), lower_gray, upper_gray)

res = cv2.bitwise_and(img, img, mask=mask)

contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

for cnt in contours : 
    cv2.drawContours(res, cnt, -1, (255,0,0), 3)

plt.figure(figsize=(20,14))
plt.subplot(1,3,1)
plt.imshow(img[:,:,::-1])
plt.title("Original Image")

plt.subplot(1,3,2)
plt.imshow(mask, cmap="gray")
plt.title("Binary Image")

plt.subplot(1,3,3)
plt.imshow(res[:,:,::-1])
plt.title("Result")

___
## 1.1.3 Visual Inspection add Contour Filtering

In [ ]:
# EXAMPLE 4 : (part_b.jpg)
# Detect Contour from Binary Image (Range Thresholding) + 
# Background removal + 
# Contour Filter by hierarchy

img = cv2.imread("part_b.jpg")

#convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Threshold the HSV image to get only gray colors
mask = cv2.inRange(hsv.copy(), lower_gray, upper_gray)

res = cv2.bitwise_and(img, img, mask=mask)

contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

for cnt, hrcy in zip(contours, hierarchy[0]):

    # Find All Extreme Outer Contour [BLUE]
    if hrcy[3] == -1 :      
        cv2.drawContours(res, cnt, -1, (255,0,0), 3)

    # Find All child contour [GREEN]
    if hrcy[3] != -1 and hrcy[2] == -1:      
        cv2.drawContours(res, cnt, -1, (0,255,0), 3)

plt.figure(figsize=(20,14))
plt.subplot(1,3,1)
plt.imshow(img[:,:,::-1])
plt.title("Original Image")

plt.subplot(1,3,2)
plt.imshow(mask, cmap="gray")
plt.title("Binary Image")

plt.subplot(1,3,3)
plt.imshow(res[:,:,::-1])
plt.title("Result")

In [ ]:
# EXAMPLE 4 : (part_b.jpg)
# Detect Contour from Binary Image (Range Thresholding) + 
# Background removal + 
# Contour Filter by hierarchy + 
# Contour Filter by Contour Property

img = cv2.imread("part_b.jpg")

#convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Threshold the HSV image to get only gray colors
mask = cv2.inRange(hsv.copy(), lower_gray, upper_gray)

res = cv2.bitwise_and(img, img, mask=mask)

contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

for cnt, hrcy in zip(contours, hierarchy[0]):
    # find contour Area & boungin Rect
    area = cv2.contourArea(cnt)
    x, y, w, h = cv2.boundingRect(cnt)

    # calculate aspectRatio & extent
    aspectRatio = float(w)/h 
    rect_area = w*h
    extent = float(area)/rect_area
    
    # filter a small contour
    if area <= 200:
        continue 

    # Find All Extreme Outer Contour [BLUE]
    if hrcy[3] == -1 :   
        # Find contour with aspec ration between 0.1~0.3 (vertical rect), with extent 0.4
        if aspectRatio < 0.3 and aspectRatio > 0.1 and extent > 0.4:   
            cv2.drawContours(res, cnt, -1, (255,0,0), 3)

    # Find All child contour [GREEN]
    if hrcy[3] != -1 :#and hrcy[2] == -1:   
        # Find contour with aspec ration between 0.5~1.5 (near square), with extent 0.4
        if aspectRatio < 1.5 and aspectRatio > 0.5 and extent > 0.4:     
            cv2.drawContours(res, cnt, -1, (0,255,0), 3)

plt.figure(figsize=(20,14))
plt.subplot(1,3,1)
plt.imshow(img[:,:,::-1])
plt.title("Original Image")

plt.subplot(1,3,2)
plt.imshow(mask, cmap="gray")
plt.title("Binary Image")

plt.subplot(1,3,3)
plt.imshow(res[:,:,::-1])
plt.title("Result")

___
## 1.1.4 Visual Inspection add Child Contour Counting for each Parent

In [ ]:
# EXAMPLE 5 : (part_b.jpg)
# Detect Contour from Binary Image (Range Thresholding) + 
# Background removal + 
# Contour Filter by hierarchy + 
# Contour Filter by Contour Property
# Count Child Contour for each parent

img = cv2.imread("part_b.jpg")

#convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Threshold the HSV image to get only gray colors
mask = cv2.inRange(hsv.copy(), lower_gray, upper_gray)

res = cv2.bitwise_and(img, img, mask=mask)

contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

object_contour = {}
object_count = {}
object_id = 0
for cnt, hrcy in zip(contours, hierarchy[0]):
    # find contour Area & boungin Rect
    area = cv2.contourArea(cnt)
    x, y, w, h = cv2.boundingRect(cnt)

    # calculate aspectRatio & extent
    aspectRatio = float(w)/h 
    rect_area = w*h
    extent = float(area)/rect_area
    
    # filter a small contour
    if area <= 200:
        continue 
    
   # Find All Extreme Outer Contour [BLUE]
    if hrcy[3] == -1 :   
        # Find contour with aspec ration between 0.1~0.3 (vertical rect), with extent 0.4
        if aspectRatio < 0.3 and aspectRatio > 0.1 and extent > 0.4:      
            cv2.drawContours(res, cnt, -1, (255,0,0), 3)
            
            object_contour["object_%d" % object_id] = cnt # insert parent contour
            object_count["object_%d" % object_id] = 0 # set initinal count 0
            object_id += 1

    # Find All child contour [GREEN]
    if hrcy[3] != -1 : #and hrcy[2] == -1:   
        # Find contour with aspec ration between 0.5~1.5 (near square), with extent 0.4
        if aspectRatio < 1.5 and aspectRatio > 0.5 and extent > 0.4: 
            cv2.drawContours(res, cnt, -1, (0,255,0), 3)

            for cnt_obj in object_contour:
                # find the child contour on wich parrent contour
                if cv2.pointPolygonTest(object_contour[cnt_obj], (x, y), measureDist=True) > 0 :
                    object_count[cnt_obj] += 1

plt.figure(figsize=(20,14))
plt.subplot(1,3,1)
plt.imshow(img[:,:,::-1])
plt.title("Original Image")

plt.subplot(1,3,2)
plt.imshow(mask, cmap="gray")
plt.title("Binary Image")

plt.subplot(1,3,3)
plt.imshow(res[:,:,::-1])
plt.title("Result")

print(object_count)

___
## 1.1.5 Visual Inspection add Draw_Ped() box & Threshold Counter

In [ ]:
# EXAMPLE 6 : (part_b.jpg)
# Detect Contour from Binary Image (Range Thresholding) + 
# Background removal + 
# Contour Filter by hierarchy + 
# Contour Filter by Contour Property
# Count Child Contour for each parent
# Draw_Ped on object

img = cv2.imread("part_b.jpg")

#convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Threshold the HSV image to get only gray colors
mask = cv2.inRange(hsv.copy(), lower_gray, upper_gray)

res = cv2.bitwise_and(img, img, mask=mask)

contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

object_contour = {}
object_count = {}
object_id = 0
threshold_count = 22 # min number of child contour 

for cnt, hrcy in zip(contours, hierarchy[0]):
    # find contour Area & boungin Rect
    area = cv2.contourArea(cnt)
    x, y, w, h = cv2.boundingRect(cnt)

    # calculate aspectRatio & extent
    aspectRatio = float(w)/h 
    rect_area = w*h
    extent = float(area)/rect_area
    
    # filter a small contour
    if area <= 200:
        continue 
    
   # Find All Extreme Outer Contour [BLUE]
    if hrcy[3] == -1 :   
        # Find contour with aspec ration between 0.1~0.3 (vertical rect), with extent 0.4
        if aspectRatio < 0.3 and aspectRatio > 0.1 and extent > 0.4:      
            cv2.drawContours(res, cnt, -1, (255,0,0), 3)
            
            object_contour["object_%d" % object_id] = cnt # insert parent contour
            object_count["object_%d" % object_id] = 0 # set initinal count 0
            object_id += 1

    # Find All child contour [GREEN]
    if hrcy[3] != -1 : #and hrcy[2] == -1:   
        # Find contour with aspec ration between 0.5~1.5 (near square), with extent 0.4
        if aspectRatio < 1.5 and aspectRatio > 0.5 and extent > 0.4:    
            cv2.drawContours(res, cnt, -1, (0,255,0), 3)

            for obj_name in object_contour:
                # find the child contour on wich parrent contour
                if cv2.pointPolygonTest(object_contour[obj_name], (x, y), measureDist=True) > 0 :
                    object_count[obj_name] += 1


for obj_name in object_count:
    x, y, w, h = cv2.boundingRect(object_contour[obj_name])
    # check if number of child contour inside parrent less than threshold count 
    if object_count[obj_name] < threshold_count :
        img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                    font_size=0.7, alpha=0.6, bg_color=(0,0,255), ouline_color=(0,0,255), text_color=(0,0,0))
    else :
        img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                    font_size=0.7, alpha=0.6, bg_color=(0,255,0), ouline_color=(0,255,0), text_color=(0,0,0))        

plt.figure(figsize=(20,30))
plt.subplot(1,2,1)
plt.imshow(img[:,:,::-1])
plt.title("Original Image")

plt.subplot(1,2,2)
plt.imshow(res[:,:,::-1])
plt.title("Result")


In [ ]:
# EXAMPLE 7 : (part_a.jpg)
# Detect Contour from Binary Image (Range Thresholding) + 
# Background removal + 
# Contour Filter by hierarchy + 
# Contour Filter by Contour Property
# Count Child Contour for each parent
# Draw_Ped on object

# define range of gray color in HSV
lower_gray = np.array([0, 0, 10])
upper_gray = np.array([180, 100, 170])


img = cv2.imread("part_a.jpg")

#convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Threshold the HSV image to get only gray colors
mask = cv2.inRange(hsv.copy(), lower_gray, upper_gray)

res = cv2.bitwise_and(img, img, mask=mask)

contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

object_contour = {}
object_count = {}
object_id = 0
threshold_count = 2 # minimum number of child contour 

for cnt, hrcy in zip(contours, hierarchy[0]):
    # find contour Area & boungin Rect
    area = cv2.contourArea(cnt)
    x, y, w, h = cv2.boundingRect(cnt)

    # calculate aspectRatio & extent
    aspectRatio = float(w)/h 
    rect_area = w*h
    extent = float(area)/rect_area
    
    # filter a small contour
    if area <= 200:
        continue 

   # Find All Extreme Outer Contour [BLUE]
    if hrcy[3] == -1 :   
        # Find contour with aspec ration between 0.3~0.5 (sort vertical rect), with extent 0.2 
        if aspectRatio < 0.5 and aspectRatio > 0.3 and extent > 0.4 :      
            cv2.drawContours(res, cnt, -1, (255,0,0), 3)
            
            object_contour["object_%d" % object_id] = cnt # insert parent contour
            object_count["object_%d" % object_id] = 0 # set initinal count 0
            object_id += 1

    # Find All child contour [GREEN]
    if hrcy[3] != -1 : #and hrcy[2] == -1:   
        # Find contour with aspec ration between 0.5~1.5 (near square), with extent 0.4 
        if aspectRatio < 1.5 and aspectRatio > 0.5 and extent > 0.4 :    
            cv2.drawContours(res, cnt, -1, (0,255,0), 3)

            for obj_name in object_contour:
                # find the child contour on wich parrent contour
                if cv2.pointPolygonTest(object_contour[obj_name], (x, y), measureDist=True) > 0 :
                    object_count[obj_name] += 1


for obj_name in object_count:
    x, y, w, h = cv2.boundingRect(object_contour[obj_name])
    # check if number of child contour inside parrent less than threshold count 
    if object_count[obj_name] < threshold_count :
        img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                    font_size=0.7, alpha=0.6, bg_color=(0,0,255), ouline_color=(0,0,255), text_color=(0,0,0))
    else :
        img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                    font_size=0.7, alpha=0.6, bg_color=(0,255,0), ouline_color=(0,255,0), text_color=(0,0,0))        

plt.figure(figsize=(20,30))
plt.subplot(1,2,1)
plt.imshow(img[:,:,::-1])
plt.title("Original Image")

plt.subplot(1,2,2)
plt.imshow(res[:,:,::-1])
plt.title("Result")


___
## 1.1.6 Visual Inspection (Child Contour Counting based on Hierarchy)
- this implementation NOT using `cv2.pointPolygonTest()` to determine the child contour inside which parent contour.
- BUT using full hierarchial TREE on detected contour

In [ ]:
# EXAMPLE 8 : (part_b.jpg)
# Detect Contour from Binary Image (Range Thresholding) + 
# Background removal + 
# Contour Filter by hierarchy + 
# Contour Filter by Contour Property
# Count Child Contour for each parent using hierarchy
# Draw_Ped on object

img = cv2.imread("part_b.jpg")

#convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Threshold the HSV image to get only gray colors
mask = cv2.inRange(hsv.copy(), lower_gray, upper_gray)

res = cv2.bitwise_and(img, img, mask=mask)

contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

object_hierarchy_id = {}
object_contour = {}
object_count = {}
object_id = 0
threshold_count = 22 # min number of child contour 

for i, (cnt, hrcy) in enumerate(zip(contours, hierarchy[0])):
    # find contour Area & boungin Rect
    area = cv2.contourArea(cnt)
    x, y, w, h = cv2.boundingRect(cnt)

    # calculate aspectRatio & extent
    aspectRatio = float(w)/h 
    rect_area = w*h
    extent = float(area)/rect_area
    
    # filter a small contour
    if area <= 200:
        continue 
    
   # Find All Extreme Outer Contour [BLUE]
    if hrcy[3] == -1 :   
        # Find contour with aspec ration between 0.1~0.3 (vertical rect), with extent 0.4
        if aspectRatio < 0.3 and aspectRatio > 0.1 and extent > 0.4:      
            cv2.drawContours(res, cnt, -1, (255,0,0), 3)
            
            object_hierarchy_id["object_%d" % object_id] = i # insert parent hierarchy id
            object_contour["object_%d" % object_id] = cnt # insert parrent contour
            object_count["object_%d" % object_id] = 0 # set initinal count 0
            object_id += 1

    # Find All child contour [GREEN]
    if hrcy[3] != -1 : #and hrcy[2] == -1:   
        # Find contour with aspec ration between 0.5~1.5 (near square), with extent 0.4
        if aspectRatio < 1.5 and aspectRatio > 0.5 and extent > 0.4:    
            cv2.drawContours(res, cnt, -1, (0,255,0), 3)

            for obj_name in object_hierarchy_id :
                # check if parent for that child contour inside the parent contour
                if hrcy[3] == object_hierarchy_id[obj_name] :
                    object_count[obj_name] += 1


for obj_name in object_count:
    x, y, w, h = cv2.boundingRect(object_contour[obj_name])
    # check if number of child contour inside parrent less than threshold count 
    if object_count[obj_name] < threshold_count :
        img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                    font_size=0.7, alpha=0.6, bg_color=(0,0,255), ouline_color=(0,0,255), text_color=(0,0,0))
    else :
        img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                    font_size=0.7, alpha=0.6, bg_color=(0,255,0), ouline_color=(0,255,0), text_color=(0,0,0))        

plt.figure(figsize=(20,30))
plt.subplot(1,2,1)
plt.imshow(img[:,:,::-1])
plt.title("Original Image")

plt.subplot(1,2,2)
plt.imshow(res[:,:,::-1])
plt.title("Result")


____
## 1.2 CUDA Implementation

In [ ]:

# load image in Host memory
img = cv2.imread("part_b.jpg")
h, w, c = img.shape

# GPU memory initialization
img_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
img_GpuMat.create((w, h), cv2.CV_8UC3) # cv2.CV_8UC3 -> 8bit image 3 channel
hsv_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
hsv_GpuMat.create((w, h), cv2.CV_8UC3) # cv2.CV_8UC3 -> 8bit image 3 channel
h_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
h_GpuMat.create((w, h), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
s_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
s_GpuMat.create((w, h), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
v_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
v_GpuMat.create((w, h), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
mask_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
mask_GpuMat.create((w, h), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
res_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
res_GpuMat.create((w, h), cv2.CV_8UC3) # cv2.CV_8UC3 -> 8bit image 3 channel

# upload to GPU memory
img_GpuMat.upload(img)

# CUDA convert to hsv
cv2.cuda.cvtColor(img_GpuMat, cv2.COLOR_BGR2HSV, hsv_GpuMat)

# split HSV GPU Mat
cv2.cuda.split(hsv_GpuMat, [h_GpuMat, s_GpuMat, v_GpuMat])

# CUDA Threshold the V(20,150) ~ HSV GPU Mat to get only gray colors
cv2.cuda.inRange(v_GpuMat, 20, 150, mask_GpuMat)

# CUDA bitwise operation
cv2.cuda.bitwise_not(img_GpuMat, res_GpuMat, mask=mask_GpuMat) # apply bitwise NOT to original image -> result image
cv2.cuda.bitwise_not(res_GpuMat, res_GpuMat, mask=mask_GpuMat) # apply bitwise NOT to result image 

# Download Matrix to Host Memory                                                               
mask = mask_GpuMat.download()
res = res_GpuMat.download()

# find contour
contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

object_hierarchy_id = {}
object_contour = {}
object_count = {}
object_id = 0
threshold_count = 22 # min number of child contour 

for i, (cnt, hrcy) in enumerate(zip(contours, hierarchy[0])):
    # find contour Area & boungin Rect
    area = cv2.contourArea(cnt)
    x, y, w, h = cv2.boundingRect(cnt)

    # calculate aspectRatio & extent
    aspectRatio = float(w)/h 
    rect_area = w*h
    extent = float(area)/rect_area
    
    # filter a small contour
    if area <= 200:
        continue 
    
   # Find All Extreme Outer Contour [BLUE]
    if hrcy[3] == -1 :   
        # Find contour with aspec ration between 0.1~0.3 (vertical rect), with extent 0.4
        if aspectRatio < 0.3 and aspectRatio > 0.1 and extent > 0.4:      
            cv2.drawContours(res, cnt, -1, (255,0,0), 3)
            
            object_hierarchy_id["object_%d" % object_id] = i # insert parent hierarchy id
            object_contour["object_%d" % object_id] = cnt # insert parrent contour
            object_count["object_%d" % object_id] = 0 # set initinal count 0
            object_id += 1

    # Find All child contour [GREEN]
    if hrcy[3] != -1 : #and hrcy[2] == -1:   
        # Find contour with aspec ration between 0.5~1.5 (near square), with extent 0.4
        if aspectRatio < 1.5 and aspectRatio > 0.5 and extent > 0.4:    
            cv2.drawContours(res, cnt, -1, (0,255,0), 3)

            for obj_name in object_hierarchy_id :
                # check if parent for that child contour inside the parent contour
                if hrcy[3] == object_hierarchy_id[obj_name] :
                    object_count[obj_name] += 1


for obj_name in object_count:
    x, y, w, h = cv2.boundingRect(object_contour[obj_name])
    # check if number of child contour inside parrent less than threshold count 
    if object_count[obj_name] < threshold_count :
        img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                    font_size=0.7, alpha=0.6, bg_color=(0,0,255), ouline_color=(0,0,255), text_color=(0,0,0))
    else :
        img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                    font_size=0.7, alpha=0.6, bg_color=(0,255,0), ouline_color=(0,255,0), text_color=(0,0,0))

plt.figure(figsize=(20,30))
plt.subplot(1,2,1)
plt.imshow(img[:,:,::-1])
plt.title("Original Image")

plt.subplot(1,2,2)
plt.imshow(res[:,:,::-1])
plt.title("Result")

____
____

# 2. Shape Detection

# 2.1 Hough Line Transform (Line Detector)

<img src="res/lane.gif" style="width:500px, margin-top:10px"></img>
- The Hough Line Transform is a transform used to detect **straight lines**.
- To apply the Transform, first an **edge detection** pre-processing is desirable.
- As you know, a line in the image space can be expressed with two variables. For example:
    - In the **Cartesian** coordinate system: Parameters: ($m$,$b$).
    - In the **Polar** coordinate system: Parameters: ($r$,$θ$). <br>
      <img src="res/Hough_Lines_Tutorial_Theory_0.jpg" style="width:400px, margin-top:10px"></img> <br>
    - For Hough Transforms, we will express lines in the *Polar system*. Hence, a line equation can be written as: <br>
    $y = \left ( -\dfrac{\cos \theta}{\sin \theta} \right ) x + \left ( \dfrac{r}{\sin \theta} \right )$ <br>
    Arranging the terms: $r = x \cos \theta + y \sin \theta$ <br>
    - In general for each point ($x_{0}, y_{0}$), we can define the family of lines that goes through that point as: <br>
        $r_{\theta} = x_{0} \cdot \cos \theta + y_{0} \cdot \sin \theta$ <br>
        Meaning that each pair ($r_{\theta},\theta$) represents each line that passes by ($x_{0}, y_{0}$).
    - If for a given ($x_{0}, y_{0}$) we plot the family of lines that goes through it, we get a sinusoid. For instance, for $x_{0}$=8 and $y_{0}$=6 we get the following plot (in a plane $θ - r$): <br>
        <img src="res/Hough_Lines_Tutorial_Theory_1.jpg" style="width:400px, margin-top:10px"></img> <br>
        We consider only points such that $r > 0$ and $0< \theta < 2 \pi$.
    - We can do the same operation above for all the points in an image. If the curves of two different points intersect in the plane $θ - r$, that means that both points belong to a same line. For instance, following with the example above and drawing the plot for two more points: $x_{1}=4, y_{1}=9$ and $x_{2}=12, y_{2}=3$, we get: <br>
        <img src="res/Hough_Lines_Tutorial_Theory_2.jpg" style="width:400px, margin-top:10px"></img> <br>
        The three plots intersect in one single point (0.925,9.6), these coordinates are the parameters (θ$,r$) or the line in which ($x_{0},y_{0}$), ($x_{1},y_{1}$) and ($x_{2},y_{2}$).<br><br>
    > A line can be detected by finding the **number of intersections between curves**.The **more curves intersecting** means that the line represented by that **intersection have more points**.
    > In general, we can define a **threshold** of the **minimum number of intersections** needed to **detect a line**.
    > This is what the **Hough Line Transform** does. It keeps track of the intersection between curves of every point in the image. 

- OpenCV implements two kind of Hough Line Transforms: <r><br>
    - The **Standard Hough Transform** <br>
        It consists in pretty much what we just explained in the previous section. It gives you as result a vector of couples ($θ,rθ$)
        In OpenCV it is implemented with the function `cv2.HoughLines()`.

    - The **Probabilistic Hough Line Transform** <br>
        A more efficient implementation of the Hough Line Transform. It gives as output the extremes of the detected lines ($x_{0},y_{0},x_{1},y_{1}$)
        In OpenCV it is implemented with the function `cv2.HoughLinesP()`.

### 2.1.1 Standard Hough Transform `cv2.HoughLines()`

- menggunakan method `cv2.HoughLines(img, rho, theta, threshold, lines, srn, stn)`
- with the following arguments:
    - `img` : input image (edge image)
    - `rho` : The resolution of the parameter r in pixels. We use 1 pixel.
    - `theta` : The resolution of the parameter θ in radians. We use 1 degree (`np.pi/180`)
    - `threshold` : The minimum number of intersections to *detect* a line
    - `lines` : A vector that will store the parameters ($r,θ$) of the detected lines
    - `srn` and `stn` : Default parameters to zero. 
- output `lines` is list with item `[[tho, theta]]`


### 2.1.2 Probabilistic  Hough Transform `cv2.HoughLinesP()`

- menggunakan method `cv2.HoughLinesP(img, res_rho, res_theta, threshold, lines, minLinLength, maxLineGap)`
- with the following arguments:
    - `img` : input image (edge image)
    - `res_rho` : The resolution of the parameter r in pixels. We use 1 pixel.
    - `res_theta` : The resolution of the parameter θ in radians. We use 1 degree (`np.pi/180`)
    - `threshold` : The minimum number of intersections to *detect* a line
    - `lines` : A vector that will store the parameters ($r,θ$) of the detected lines
    - `minLinLength` : The minimum number of points that can form a line. Lines with less than this number of points are disregarded.
    - `maxLineGap` : The maximum gap between two points to be considered in the same line.
- output `lines` is list with item `[[tho, theta]]`

In [ ]:
# EXAMPLE 1 : Hough Lines

img = cv2.imread('road.jpg')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 200)

lines = cv2.HoughLines(edges, 1, np.pi / 180, 100, None, 0, 0)

if lines is not None :
    for rho, theta in lines[:, 0, :]:
        m = - 1 * np.cos(theta) / np.sin(theta)
        c = rho / np.sin(theta)
        
        x0, x1 = 0, 1000
        y0 = int(m*x0 + c)
        y1 = int(m*x1 + c)
        cv2.line(img, (x0, y0), (x1, y1), (255, 0, 0), 1, cv2.LINE_AA)

plt.figure(figsize=(20,30))
plt.subplot(1,2,1)
plt.imshow(img[:,:,::-1])
plt.title("Hough Lines")

In [ ]:
# EXAMPLE 2 : Hough Lines Probabilistic

img = cv2.imread('road.jpg')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 200)
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 50, minLineLength=50, maxLineGap=30)

if lines is not None : 
    for x1, y1, x2, y2 in lines[:, 0, :]:
        cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 1, cv2.LINE_AA)

plt.figure(figsize=(20,30))
plt.subplot(1,2,1)
plt.imshow(img[:,:,::-1])
plt.title("Hough LinesP")

___
## 2.5 CUDA Implementation Hough Lines
- Class `cv2.cuda.createHoughLinesDetector(res_rho, res_theta, threshold, doSort, maxLines)`
- Where : 
    - `res_rho` : Distance resolution of the accumulator in pixels.
    - `res_theta` : Angle resolution of the accumulator in radians.
    - `threshold` : The minimum number of intersections to *detect* a line.
    - `doSort` : Performs lines sort by votes.
    - `maxLines` : Maximum number of output lines.
- Method to finds lines in a binary image `.detect(src, d_lines)`
- Where : 
    - `src` : GPU Mat CV_8UC1 (8 bit 1 channel)
    - `d_lines` : GPU Mat vector of lines in GPU memory.

In [ ]:
# EXAMPLE 3 : CUDA Hough Lines

img = cv2.imread('road.jpg')
h, w, c = img.shape

# GPU memory initialization
img_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
img_GpuMat.create((w, h), cv2.CV_8UC3) # cv2.CV_8UC3 -> 8bit image 3 channel
gray_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
gray_GpuMat.create((w, h), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
edged_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
edged_GpuMat.create((w, h), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel

# Define CUDA Canny Edge Detection & Hough Lines Object
Canny = cv2.cuda.createCannyEdgeDetector(50, 200) # Initialize Canny Detector in CUDA
HoughLines = cv2.cuda.createHoughLinesDetector(1, np.pi/180, 100, True, 100)

# upload to GPU memory
img_GpuMat.upload(img)

# CUDA convert to Grayscale
cv2.cuda.cvtColor(img_GpuMat, cv2.COLOR_BGR2GRAY, gray_GpuMat)

# Apply Canny Object Detection using CUDA
Canny.detect(gray_GpuMat, edged_GpuMat)

# Apply Hough Lines Detection
d_lines = HoughLines.detect(edged_GpuMat)

# Download Lines to Host Memory
lines = d_lines.download()

# draw line
if lines is not None :
    for rho, theta in lines[0, :, :]:
        m = - 1 * np.cos(theta) / np.sin(theta)
        c = rho / np.sin(theta)
        
        x0, x1 = 0, 1000
        y0 = int(m*x0 + c)
        y1 = int(m*x1 + c)
        cv2.line(img, (x0, y0), (x1, y1), (255, 0, 0), 1, cv2.LINE_AA)

plt.figure(figsize=(20,30))
plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Hough Lines")

___
## 2.2 Hough Circle Transform
- The Hough Circle Transform works in a roughly analogous way to the Hough Line Transform explained in the previous tutorial.
- In the line detection case, a line was defined by two parameters ($r,θ$). 
- Parameters to define a circle:
$C : ( x_{center}, y_{center}, r )$
- where ($x_{center},y_{center}$) define the center position (green point) and $r$ is the radius, which allows us to completely define a circle, as it can be seen below:<br>
 <img src="res/Hough_Circle_Tutorial_Theory_0.jpg" style="width:400px, margin-top:10px"></img><br><br>
- menggunakan method `cv2.HoughCircles(img, mode, dp, min_dist_center, param1, param2, min_radius, max_radius)`
- with the arguments:
    - `img` : input image.
    - `mode` : 
        - `cv2.HOUGH_STANDARD` : Classical or standard Hough transform.
        - `cv2.HOUGH_PROBABILISTIC` : Probabilistic Hough transform (more efficient in case if the picture contains a few long linear segments).
        - `cv2.HOUGH_MULTI_SCALE` : multi-scale variant of the classical Hough transform. 
        - `cv2.HOUGH_GRADIENT`
    - `dp` : The inverse ratio of resolution (default 1).
    - `min_dist_center` : Minimum distance between detected centers.
    - `param1` : Upper threshold for the internal Canny edge detector.
    - `param2` : Threshold for center detection.
    - `min_radius` : Minimum radius to be detected. If unknown, put zero as default.
    - `max_radius` : Maximum radius to be detected. If unknown, put zero as default.

In [25]:
# EXAMPLE 4 : Hough Circle

img = cv2.imread('eye.jpg')
h, w, c = img.shape

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

blur = cv2.GaussianBlur(gray,(5,5), 0, 0)

circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, 1, h/64, param1=200, param2=17, minRadius=21, maxRadius=30)

if circles is not None :
    for x, y, r in np.uint0(circles[0, :, :]):
        cv2.circle(img, (x, y), r, (0, 255, 0), 2)
        
plt.figure(figsize=(20,30))
plt.subplot(1,2,1)
plt.imshow(img[:,:,::-1])
plt.title("Hough Circle")

error: OpenCV(4.5.3) /tmp/build_opencv/opencv/modules/imgproc/src/hough.cpp:2318: error: (-5:Bad argument) Unrecognized method id. Actually only CV_HOUGH_GRADIENT is supported. in function 'HoughCircles'


___
## 2.5 CUDA Implementation Hough Circle
- Class `cv2.cuda.createHoughCirclesDetector(dp, min_dist_center, param1, param2, minRadius, maxRadius)`
- Where : 
    - `dp` : The inverse ratio of resolution (default 1).
    - `min_dist_center` : Minimum distance between detected centers.
    - `param1` : Upper threshold for the internal Canny edge detector.
    - `param2` : Threshold for center detection.
    - `min_radius` : Minimum radius to be detected. If unknown, put zero as default.
    - `max_radius` : Maximum radius to be detected. If unknown, put zero as default.
- Method to finds circles  in a binary image `.detect(src, d_circles)`
- Where : 
    - `src` : GPU Mat CV_8UC1 (8 bit 1 channel)
    - `d_circles` : GPU Mat vector of circles in GPU memory.

In [ ]:
# EXAMPLE 5 : CUDA Hough Circle

img = cv2.imread('eye.jpg')
h, w, c = img.shape

# GPU memory initialization
img_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
img_GpuMat.create((w, h), cv2.CV_8UC3) # cv2.CV_8UC3 -> 8bit image 3 channel
gray_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
gray_GpuMat.create((w, h), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
res_Gaussian_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
res_Gaussian_GpuMat.create((w, h), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel

# Create CUDA Gaussian Filter Object with kernel size 7x7
cuda_GaussianFilter = cv2.cuda.createGaussianFilter(cv2.CV_8UC1, cv2.CV_8UC1, (5,5), 0, 0)
# Define CUDA Hough Circles Object
HoughCircles = cv2.cuda.createHoughCirclesDetector(1, h/64, 200, 17, 21, 30)

# upload to GPU memory
img_GpuMat.upload(img)

# CUDA convert to Grayscale
cv2.cuda.cvtColor(img_GpuMat, cv2.COLOR_BGR2GRAY, gray_GpuMat)

# apply CUDA Gaussian Filter
cuda_GaussianFilter.apply(gray_GpuMat, res_Gaussian_GpuMat)

# Apply Hough Circles Detection
d_circles = HoughCircles.detect(res_Gaussian_GpuMat)

# Download Circles to Host Memory
circles = d_circles.download()

# draw circle
if circles is not None :
    for x, y, r in np.uint0(circles[0, :, :]):
        cv2.circle(img, (x, y), r, (0, 255, 0), 2)
        
plt.figure(figsize=(20,30))
plt.subplot(1,2,1)
plt.imshow(img[:,:,::-1])
plt.title("Hough Circle")

___
## 2.3 Hough Transform Application (Hough Circle) - Real-time Circle Detection

In [ ]:
# EXAMPLE 6 : Detect Yellow Ball using Hough Circles

cap = cv2.VideoCapture("yellow_ball.mp4")


while cap.isOpened():
    ret, frame = cap.read()
    if not ret :
        break 

    frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)
    h, w, c = frame.shape

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, h/2, param1=180, param2=17, minRadius=21, maxRadius=100)

    if circles is not None :
        for x, y, r in np.uint0(circles[0, :, :]):
            cv2.circle(frame, (x, y), r, (0, 255, 0), 2)
    
    cv2.imshow('Hough Circle - Video', frame)

    if cv2.waitKey(10) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

- CUDA Version for Detect Yellow Ball

In [ ]:
# EXAMPLE 7 : Detect Yellow Circle using CUDA Hough Circles

cap = cv2.VideoCapture("yellow_ball.mp4")

h, w, c = cap.read()[1].shape
h, w = h//2, w//2

# GPU memory initialization
img_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
img_GpuMat.create((w, h), cv2.CV_8UC3) # cv2.CV_8UC3 -> 8bit image 3 channel
gray_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
gray_GpuMat.create((w, h), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel

# Define CUDA Hough Circles Object
HoughCircles = cv2.cuda.createHoughCirclesDetector(1, h/2, 180, 17, 21, 100)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret :
        break

    frame = cv2.resize(frame, (w,h))

    # upload to GPU memory
    img_GpuMat.upload(frame)
    
    # CUDA convert to Grayscale
    cv2.cuda.cvtColor(img_GpuMat, cv2.COLOR_BGR2GRAY, gray_GpuMat)

    # Apply Hough Circles Detection
    d_circles = HoughCircles.detect(gray_GpuMat)

    # Download Circles to Host Memory
    circles = d_circles.download()

    # draw circle
    if circles is not None :
        for x, y, r in np.uint0(circles[0, :, :]):
            cv2.circle(frame, (x, y), r, (0, 255, 0), 2)

    cv2.imshow('Hough Circle - Video', frame)
    if cv2.waitKey(10) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

___
## 2.4 Real-tim Road Line Detection <br>
Original Video : [https://www.youtube.com/watch?v=KWJaBJYJIjI](https://www.youtube.com/watch?v=KWJaBJYJIjI)

In [ ]:
maxLineGap = 200
minLineLength = 50
title_window = "Road Lane Detector"

# define ROI polygon in video frame with size 720x1280
dim = [720, 1280]
roi_poly = np.array([[ 118,  678],
                    [ 586,  434],
                    [ 868,  420],
                    [1226,  650]])

# find fraction ROI polygon (0-1)
f_roi_poly = roi_poly/dim
f = 0.5

# load video file
cap = cv2.VideoCapture('drive.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret :
        break # stop loop

    # resize frame
    frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)
    dim = frame.shape[:2]

    # convert to gray
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # create masking image
    mask = np.zeros(gray.shape).astype(np.uint8)
    
    # draw polygone in new frame size
    pts = f_roi_poly*dim
    cv2.fillPoly(mask, [pts.astype(np.int64)], (255,255,255))
    
    # apply bitwise operation
    roi = cv2.bitwise_and(gray, gray, mask=mask)

    # apply simple thresholding 
    ___, thresh = cv2.threshold(roi, 130, 255, cv2.THRESH_BINARY)

    # apply Canny edge detection
    edged = cv2.Canny(thresh, 127, 200)

    # find lines using Hough Line Probabilistic
    lines = cv2.HoughLinesP(edged, 1, np.pi/180, 10, None, minLineLength=20, maxLineGap=250)

    # draw lines
    if lines is not None:
        for x1, y1, x2, y2 in lines[:,0,:]:
            cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 255), 1, cv2.LINE_AA)

    cv2.imshow('roi', roi)
    cv2.imshow(title_window, frame)

    if cv2.waitKey(25) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

- Dari hasil sebelumnya kita dapat mendeteksi Road Lane dengan menggunakan `cv2.HoughLinesP()`
- Namun banyak `lines` yang terdeteksi disisi kanan maupun  kiri,<br>
<img src="res/overlay_v1.png" style="width: 500px; margin-top:10px;" > </img> <br>
<img src="res/slope_intersect.png" style="width: 400px; margin-top:10px;" > </img> <br>
- Rata-ratakan slope ($m$) dan Intersept ($c$), disisi kanan dan kiri,<br>
<img src="res/slope_intersect_mean.png" style="width: 400px; margin-top:10px;" > </img>
- Setelahnya tentukan ($x_1, y_1$) dan ($x_2, y_2$) sisi kanan dan kiri,<br>
<img src="res/coordinate.png" style="width: 500px; margin-top:10px;" > </img>
- Tambahkan overlay sebagai berikut dari `lines` yang didapatkan  `cv2.HoughLinesP()` <br>
<img src="res/overlay.png" style="width: 500px; margin-top:10px;" > </img> 


In [ ]:
r_m, l_m, r_c, l_c = [],[],[],[]
def draw_lines(shape, lines, thickness=3, margin_top = 0.65):
    global r_m, l_m, r_c, l_c
    h, w = shape
    overlay = np.zeros((h, w, 3), dtype=np.uint8)
    
    if lines is not None :
        for x1, y1, x2, y2 in lines[:,0,:]:

            m = (y1-y2)/(x1-x2)
            c = y2 - (m*x2)
            if m > 0.3:
                r_m.append(m)
                r_c.append(c)
            elif m < -0.3:
                l_m.append(m)
                l_c.append(c)

    # find average m and c for left and right line for the last 30 data.
    avg_l_m = np.mean(l_m[-30:])
    avg_l_c = np.mean(l_c[-30:])
    avg_r_m = np.mean(r_m[-30:])
    avg_r_c = np.mean(r_c[-30:])

    # delete data
    del l_m[:-30]
    del l_c[:-30]
    del r_m[:-30]
    del r_c[:-30]

    try:
        y1, y2 = int(margin_top*h), h
        l_x1 = int((y1 - avg_l_c)/avg_l_m)
        l_x2 = int((y2 - avg_l_c)/avg_l_m)
        r_x1 = int((y1- avg_r_c)/avg_r_m)
        r_x2 = int((y2 - avg_r_c)/avg_r_m)
        
        pts = np.array([[l_x1, y1],[l_x2, y2],[r_x2, y2],[r_x1, y1]]).astype(np.int32)
        pts = pts.reshape((-1,1,2))
        cv2.fillPoly(overlay, [pts], (0,127,50))
        cv2.line(overlay, (l_x1, y1), (l_x2, y2), (0,255,255), thickness)
        cv2.line(overlay, (r_x1, y1), (r_x2, y2), (0,255,255), thickness)
        return overlay
    except ValueError:
        pass


In [ ]:
maxLineGap = 200
minLineLength = 50
title_window = "Road Lane Detector"

# define ROI polygon in video frame with size 720x1280
dim = [720, 1280]
roi_poly = np.array([[ 118,  678],
                    [ 586,  434],
                    [ 868,  420],
                    [1226,  650]])

# find fraction ROI polygon (0-1)
f_roi_poly = roi_poly/dim
f = 0.5

# load video file
cap = cv2.VideoCapture('drive.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret :
        break # stop loop

    # resize frame
    frame = cv2.resize(frame, (0,0), fx=0.25, fy=0.25)
    dim = frame.shape[:2]

    # convert to gray
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # create masking image
    mask = np.zeros(gray.shape).astype(np.uint8)
    
    # draw polygone in new frame size
    pts = f_roi_poly*dim
    cv2.fillPoly(mask, [pts.astype(np.int64)], (255,255,255))
    
    # apply bitwise operation
    roi = cv2.bitwise_and(gray, gray, mask=mask)

    # apply simple thresholding 
    ___, thresh = cv2.threshold(roi, 130, 255, cv2.THRESH_BINARY)

    # apply Canny edge detection
    edged = cv2.Canny(thresh, 127, 200)

    # find lines using Hough Line Probabilistic
    lines = cv2.HoughLinesP(edged, 1, np.pi/180, 10, None, minLineLength=20, maxLineGap=250)

    overlay = draw_lines(edged.shape, lines, thickness = 3, margin_top = 0.65)
    frame = cv2.addWeighted(frame, 1, overlay, 0.7, 0)

    cv2.imshow(title_window, frame)
    cv2.imshow(title_window + "- ROI", roi)
    cv2.imshow(title_window + "- Edge", edged)
    cv2.imshow(title_window + "- Thresh", thresh)
    
    if cv2.waitKey(25) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()